# IMPORT

In [3]:
%reset -f
import numpy as np
# from scipy.constants import h, c, e, pi
# from scipy.linalg import inv
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from fractions import Fraction
import traceback
import json

In [4]:
##### ##### ##### ##### ##### ##### #####
exp  = lambda rad: np.exp(rad)
sin  = lambda rad: np.sin(rad)
cos  = lambda rad: np.cos(rad)
tan  = lambda rad: np.tan(rad)
asin = lambda rad: np.arcsin(rad)
acos = lambda rad: np.arccos(rad)
#
sqrt   = lambda vec: np.sqrt(vec)
norm   = lambda vec: np.linalg.norm(vec)
square = lambda vec: np.power(vec, 2)
##### ##### ##### ##### ##### ##### #####
def vec(*args): return np.array([*args])
def uvec(*args): 
    vec = np.array([*args])
    return  vec / np.linalg.norm(vec)
#
def prod(RES):
    INF = np.where(np.isinf(RES))
    RES[INF] = 0
    PRD = np.prod(RES, axis=1)
    if INF[0].size != 0: PRD[INF[0]] = inf
    return PRD
##### ##### ##### ##### ##### ##### #####
pi   = np.pi
inf  = np.inf
#
eps0 = 8.854187817E-12

In [5]:
def cube3d(arr, dim=0b001):
    if dim == 0b100:   return np.transpose(np.array(arr),(0, 2, 1))[:,:,::-1]
    elif dim == 0b010: return np.transpose(np.array(arr),(2, 0, 1))[::-1,:,::-1]
    elif dim == 0b001: return np.transpose(np.array(arr),(1, 2, 0))[:,:,::-1]
    else: return

def cuberange(X, Y, Z):
    for i in range(X):
        for j in range(Y):
            for k in range(Z):
                yield (i, j, k)

def span(eigenValues, eigenVector):
    return np.tensordot(eigenValues, eigenVector, axes=0)

In [6]:
def PIticks(start, end, step):
    v = np.arange(start, end+step, step)
    txt = []
    for ii in v:
        f = Fraction(ii)
        if f==0:
            txt.append(0)
            continue
        #
        if   f.numerator ==  1: num = 'π'
        elif f.numerator == -1: num = '-π'
        else: num = f'{f.numerator}π'
        #
        if f.denominator == 1: txt.append(num)
        else : txt.append(f'{num}/{f.denominator}')
    return v*pi, txt

## import

In [7]:
# https://lampz.tugraz.at/~hadley/ss1/crystaldiffraction/atomicformfactors/formfactors.php
file_path = "./JSON/AFF.json"  # 파일 경로
with open(file_path, "r") as f:
	AtomicFormFactor = json.load(f)

---
# Class

In [12]:
class Xray():
    CuKa1 = 1.5406
    def __init__(self, wavelength = 1.5406):
        self.wavelength = wavelength # Å unit
        self.K = 2 * pi / wavelength
        (h, c, e) = (6.62607015E-34, 299792458, 1.6021773349E-19)
        self.Energy = h * c / (wavelength * 1E-10) / e
    def __call__(self, *args):
        return self.K
    def tthetas2dks(self, tthetas):
        return 2 * self.K * sin(tthetas/2)

In [10]:
class Atom:
    def __init__(self, Z, def_name=None):
        self.Z = Z
        self.__denominator__ = 1
        self.f0 = None
        if def_name is None:
            (_, _, _, text) = traceback.extract_stack()[-2]
            def_name = text[:text.find('=')].strip()
        self.def_name = def_name

    def __truediv__(self, num):
        atom = Atom(self.Z / num, def_name=self.def_name)
        atom.__denominator__ /= num
        return atom

    def aff(self, G):
        if not self.f0:
            self.f0 = AtomicFormFactor[self.def_name]
        (a1, b1, a2, b2, a3, b3, a4, b4, c) = self.f0
        f0 = sum(c + vec(*[a * exp(-1 * b * np.power(G / (4 * pi), 2)) for a, b in zip(vec(a1, a2, a3, a4), vec(b1, b2, b3, b4))]))
        return f0 / self.__denominator__

In [14]:
class Molecule():
    def __init__(self, structure, abc, def_name = None):
        self.structure = structure
        self.abc = vec(*abc)
        if def_name == None:
            (_,_,_,text)=traceback.extract_stack()[-2]
            def_name  = text[:text.find('=')].strip()
        self.def_name = def_name
    def __call__(self, *args):
        if len(args)>0:
            return {self.def_name: vec(*args)}
        return self.def_name
    #
    def perovskite(A, B, O): # Atoms
        A /= 8
        O /= 2
        return cube3d([
            [[A, 0, A], [0, O, 0], [A, 0, A]],
            [[0, O, 0], [O, B, O], [0, O, 0]],
            [[A, 0, A], [0, O, 0], [A, 0, A]]
        ], 0b100)
    def pseudocubic(*abc):
        a, b, c = abc
        ac      = np.sqrt(a**2 + b**2) / 2
        cc      = c / 2
        return np.array((ac, ac, cc))
    def const_volume_abc(self, film_abc):
        a, b, _ = self.abc
        return np.array([a, b, np.prod(film_abc) / (a*b)])
    def strain(self, film):
        return Molecule(
            structure = film.structure,
            abc = self.const_volume_abc(film.abc)
        )
    def __truediv__(self, substrate):
        return Molecule(
            structure = self.structure,
            abc = substrate.const_volume_abc(self.abc),
            def_name  = self.def_name + '/' + substrate.def_name
    )
    #
    def GG2HKL(self, GG, nref=vec(0,0,1)):
        nref = uvec(*nref)
        QQ   = span(GG, nref)
        # Q * abc = 2 * pi * hkl
        HKL = QQ * self.abc / 2 / pi
        return HKL

In [16]:
class Film():
    def __init__(self, molecule, NaNbNc, configuration = []):
        self.molecule = molecule
        self.NaNbNc = NaNbNc
        self.Na, self.Nb, self.Nc = NaNbNc
        #
        self.configuration = configuration
        
    def __call__(self, *args):
        if len(args)>0:
            return self.molecule(*args)
        return self.molecule()

    def __truediv__(self, substrate):
        return Film(self.molecule, self.NaNbNc, [*self.configuration, substrate])
    
class Sample():
    def __init__(self, top_film):
        if top_film.configuration == []:
            self.films = [top_film]
        else:
            self.films = [top_film, *top_film.configuration]
        self.bulk = self.films.pop()
    def __call__(self):
        return vec(*[x() for x in self.films], self.bulk())

In [18]:
class Detector:
    def __init__(self, xray, atom=[], molecule=[], film=[], sample=[]):
        if str(atom.__class__) == "<class '__main__.Atom'>": self.atom = vec(atom)
        else : self.atom = vec(*atom)
        #
        if str(molecule.__class__) == "<class '__main__.Molecule'>": self.molecule = vec(molecule)
        else: self.molecule = vec(*molecule)

        if str(film.__class__) == "<class '__main__.Film'>": self.film = vec(film)
        else: self.film = vec(*film)

        self.sample   = sample
        self.HKL = {}
        #
        self.xray     = xray
        self.TTHETA   = np.linspace(1E-10, pi, 1000)
        self.DEGREE   = np.rad2deg(self.TTHETA)
        self.DK       = xray.tthetas2dks(self.TTHETA)
        self.nref     = {}
        self.loc      = (1, 1, 1)

    def align(self, nref):
        # nref = {'sto': (0, 0, 1)}  또는  sto(0, 0, 1)
        if str(nref.__class__) == "<class 'dict'>":
            self.nref = nref
        # nref = [sto(0,0,1), nno(0,0,2)]
        elif str(nref[0].__class__) == "<class 'dict'>":
            self.nref = dict((key, value) for dictionary in nref for key, value in dictionary.items())
        else: 
            print('ERROR_')
            print("EX) align(nref=Molecule(0,0,1))")
            print("EX) align(nref=Film(0,0,1))")

    def tthetaChange(self, ttheta):
        self.TTHETA = vec(*ttheta)
        self.DEGREE   = np.rad2deg(self.TTHETA)
        self.DK     = self.xray.tthetas2dks(vec(*ttheta))
    def degreeChange(self, start, end, length):
        self.tthetaChange(np.linspace(np.deg2rad(start), np.deg2rad(end), length))

    def errorCheck(self):
        if not len(self.nref.keys()):
            print("ERROR_")
            print("You need to set the scan orientation first.")
            print("EX) Detector.align(self, nref)")
            return True
        else: return False
    
    # Atomic Form Factor
    def AFF(self, REF=True):
        res = []
        for atom in self.atom:
            if REF:
                res.append(atom.aff(self.DK))
                # (a1, b1, a2, b2, a3, b3, a4, b4, c) = AtomicFormFactor[atom.def_name]
                # aff = sum(c + vec(*[a * exp(-1 * b * np.power(self.DK/(4*pi), 2)) for a, b in zip(vec(a1, a2, a3, a4), vec(b1, b2, b3, b4))]))
                # res.append(aff / atom.denominator)
            else:
                res.append(atom.Z / np.power(self.DK, 2))
        if self.atom.size == 1: return res[0]
        else : return vec(*res)
        
    # Structure Factor
    def SF(self, REF = True):
        if self.errorCheck(): return
        res = []
        for molecule in self.molecule:
            structure = molecule.structure 
            self.HKL[molecule()] = molecule.GG2HKL(self.DK, nref=self.nref[molecule()])
            # G = np.linalg.norm(2 * pi * hkls / molecule.abc, axis = 1)
            (Nx, Ny, Nz) = len(structure[:,0,0]),len(structure[0,:,0]),len(structure[0,0,:])
            sf = np.zeros(len(self.HKL[molecule()]))
            for (ii,jj,kk) in cuberange(Nx,Ny,Nz):
                if structure[ii,jj,kk]==0 : continue
                # rj는 sample의 abc 프레임 기준..
                rj = vec(ii,jj,kk) / vec(Nx-1, Ny-1, Nz-1)
                #
                self.atom = vec(structure[ii,jj,kk])
                sf = sf + self.AFF(REF) * exp(2j * pi * self.HKL[molecule()] @ rj)
            res.append(sf)
        if self.molecule.size == 1: return res[0]
        return vec(*res)
    
    # Scattering Factor E(R) && Slit fuNction
    def ER(self, SLIT = False):
        if self.errorCheck(): return
        N1 = vec(*self.loc)
        res = []
        for film in self.film:
            molecule = film.molecule
            self.HKL[molecule()] = molecule.GG2HKL(self.DK, nref=self.nref[molecule()])
            N2  = vec(*film.NaNbNc)
            #
            IX  = 2j * pi * self.HKL[molecule()]
            # N2 == inf check. (BULK)
            N3  = np.where(N2==inf, 0, N2)
            NUM = np.where(N2==inf, -exp(IX), exp(IX*N1)*(1-exp(IX*N3))  )
            # denominator == 0 check.
            DEN = np.where(np.abs(1-exp(IX)) > 1E-10, 1-exp(IX), 1)
            RES = np.where(np.abs(1-exp(IX)) > 1E-10, NUM/DEN, N2)
            #
            if SLIT:
                res.append(prod(RES))
            else:
                self.molecule = vec(molecule)
                SN  = prod(RES)
                SF  = self.SF(REF = True)
                # prod(RES) * np.where(np.isinf(RES), 1+1j, self.SF(REF=True))
                res.append(SN * np.where(np.isinf(SN), 1+1j, SF))
        if self.film.size == 1: 
            # lloc 말이 안돼. STO격자 N=10까지 하면, NNO격자는 N=10부터일수가 없다. HKL이 달라서.
            # nref 방향을 쌓아야하는데. 또 STO001 위에 NNO110 이런거 있으면 어떡하지??
            # 일단 보류.
            self.loc = N1 + N2 * vec(0,0,1)
            return res[0]
        return vec(*res)

	# Scattering Factor E(R)
    def SN(self):
        return self.ER(SLIT = True)


---
# Usage

In [13]:
# Usage
CuKa1 = Xray(Xray.CuKa1)
#
Sr = Atom(38)
Ti = Atom(22)
O = Atom(8)
Ru = Atom(44)
Nd = Atom(60)
Ni = Atom(28)
Ga = Atom(31)


In [15]:
# Usage
sto = Molecule(
    structure = Molecule.perovskite(Sr, Ti, O),
    abc = (3.905, 3.905, 3.905)
)
sro = Molecule(
    structure = Molecule.perovskite(Sr, Ru, O),
    abc = Molecule.pseudocubic(5.567, 5.5304, 7.8446)
)
nno = Molecule(
    structure = Molecule.perovskite(Nd, Ni, O),
    abc = Molecule.pseudocubic(5.387, 5.383, 7.610)
)
ngo = Molecule(
    structure = Molecule.perovskite(Nd, Ni, O),
    abc = Molecule.pseudocubic(5.428, 5.498, 7.708)
)
sro_sto = sro/sto # sto.strain(sro)
nno_sto = nno/sto # sto.strain(nno)
ngo_sto = ngo/sto # sto.strain(ngo)

In [19]:
# Usage
STO = Film(sto, (10, 10, 100))
SRO = Film(sro, (10, 10, 10))
NNO = Film(nno, (10, 10, 10))
NGO = Film(ngo, (10, 10, 10))
#
SRO_sto = Film(sro/sto, (10, 10, 100))
NNO_sto = Film(nno/sto, (10, 10, 100))
##
SRO_STO = Sample(SRO_sto/STO)
NNO_SRO_STO = Sample(NNO_sto/SRO_sto/STO)

# SAMP = Sample(SRO)
print('Total Configuration )')
print(NNO_SRO_STO())
print(f'BULK )  {NNO_SRO_STO.bulk()}')
print(f'FILM )  {vec(*[x() for x in NNO_SRO_STO.films])}')

Total Configuration )
['nno/sto' 'sro/sto' 'sto']
BULK )  sto
FILM )  ['nno/sto' 'sro/sto']
